## Impute Missing Values

This data set contains many columns with missing values. This notebook deals with accounting for missing values. Note that all of these commands were added to the file, `src/load_data-02.r` so that in subsequent notebooks the data is loaded via script.

There is a challenge to handling these missing values specific to the dataset. The file `doc/data_description.txt` contains a detailed description of each feature in this data set. Here, we see the description for `MasVnrType`.

    MasVnrType: Masonry veneer type

           BrkCmn	Brick Common
           BrkFace	Brick Face
           CBlock	Cinder Block
           None	None
           Stone	Stone
           
Note that one attribute for this feature is the value `None` meaning that the house in question does not have a Veneer. This is common in the data set. Unfortunately, upon loading the data, these `None` values will be taken to mean that the data is missing when they should be taken to mean `None`. Adding further complication to this, is the fact that there are features that contain actual missing values.           

In [7]:
test_df = read.csv('data/test.csv')

In [8]:
source('src/load_data-01.r')

ERROR: Error in `$<-.data.frame`(`*tmp*`, MSSubClass, value = structure(integer(0), .Label = character(0), class = "factor")): replacement has 0 rows, data has 418


In [2]:
dim(housing_df)

[1] 1460   81

In [3]:
head(housing_df)

X,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,Utilities,LotConfig,LandSlope,⋯,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Reg,Lvl,AllPub,Inside,Gtl,⋯,0,,,,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Reg,Lvl,AllPub,FR2,Gtl,⋯,0,,,,0,5,2007,WD,Normal,181500
3,60,RL,68,11250,IR1,Lvl,AllPub,Inside,Gtl,⋯,0,,,,0,9,2008,WD,Normal,223500
4,70,RL,60,9550,IR1,Lvl,AllPub,Corner,Gtl,⋯,0,,,,0,2,2006,WD,Abnorml,140000
5,60,RL,84,14260,IR1,Lvl,AllPub,FR2,Gtl,⋯,0,,,,0,12,2008,WD,Normal,250000
6,50,RL,85,14115,IR1,Lvl,AllPub,Inside,Gtl,⋯,0,,MnPrv,Shed,700,10,2009,WD,Normal,143000


### Impute `nan` Values

We begin by investigating the dataset for `nan` or "not a number" values. This value was added to numerical features with missing data. These should be taken as actual missing values.

In [4]:
nan_sums = colSums(is.na(housing_df))
nan_sums[nan_sums > 0]

LotFrontage  MasVnrArea GarageYrBlt 
        259           8          81

In [9]:
nan_sums = colSums(is.na(test_df))
nan_sums[nan_sums > 0]

Age Fare 
  86    1

We will impute the missing values by simply signing to them the mean of the extant values. There is a bit of a trick to this. As shown below taking the mean of a feature with missing values will return a `nan`.

In [5]:
 mean(housing_df$LotFrontage)

[1] NA

This can be by passing the argument `na.rm=T` to the `mean()` function.

In [6]:
mean(housing_df$LotFrontage, na.rm=T)

[1] 70.04996

Here we assign these mean values to variables.

In [7]:
mean_LotFrontage <- mean(housing_df$LotFrontage, na.rm=T)
mean_MasVnrArea <- mean(housing_df$MasVnrArea, na.rm=T)
mean_GarageYrBlt <- mean(housing_df$GarageYrBlt, na.rm=T)

Next, we use the `is.na` function to create a mask for the missing values. We then assign to the missing values the calculated mean value.

In [8]:
housing_df$LotFrontage[is.na(housing_df$LotFrontage)] <- mean_LotFrontage
housing_df$MasVnrArea[is.na(housing_df$MasVnrArea)] <- mean_MasVnrArea
housing_df$GarageYrBlt[is.na(housing_df$GarageYrBlt)] <- mean_GarageYrBlt

In [9]:
nan_sums = colSums(is.na(housing_df))
nan_sums[nan_sums > 0]

named numeric(0)

### Handling `None` Values

Here, we write a set of helper functions to help us to identify the `None` values for our categorical features. Note, that they are currently being stored as simply an empty string. We use masking by creating a mask for empty string values.

In [10]:
count_empty_values <- function (feature) {
    empty_string_mask = housing_df[feature] == ""
    return(length(housing_df[feature][empty_string_mask]))
}

count_empty_total <- function (){
    for (feature in colnames(housing_df)) {
        empty_count <- count_empty_values(feature)
        if (empty_count > 0) {
            print(paste(feature, empty_count))        
        }
    }
}

In [11]:
count_empty_total()

[1] "Alley 1369"
[1] "MasVnrType 8"
[1] "BsmtQual 37"
[1] "BsmtCond 37"
[1] "BsmtExposure 38"
[1] "BsmtFinType1 37"
[1] "BsmtFinType2 38"
[1] "Electrical 1"
[1] "FireplaceQu 690"
[1] "GarageType 81"
[1] "GarageFinish 81"
[1] "GarageQual 81"
[1] "GarageCond 81"
[1] "PoolQC 1453"
[1] "Fence 1179"
[1] "MiscFeature 1406"


Here, we note that missing values for one of the features should be taken to mean `nan`

    Electrical: Electrical system

           SBrkr	Standard Circuit Breakers & Romex
           FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
           FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
           FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
           Mix	Mixed

To handle for this, we create two lists one where an empty string signifies `None` and the other where the empty string signifies `nan`.

In [12]:
empty_means_without <-c("Alley","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1",
                        "BsmtFinType2", "FireplaceQu","GarageType","GarageFinish",
                        "GarageQual","GarageCond","PoolQC","Fence","MiscFeature","MasVnrType")

empty_means_NA <- c("Electrical")

We then right a series of helper functions to use the masking on on empty string to properly handle all of these empty strings. Note that where an empty string signifies `None` we assigned the value `"without"`.

In [13]:
replace_empty_with_without <- function(feature) {
    levels(feature) <- c(levels(feature), "without")
    empty_string_mask <- feature == ''
    feature[empty_string_mask] <- "without"
    return(feature)
}

replace_empty_with_NA <- function(feature) {
    levels(feature) <- c(levels(feature), NA)
    empty_string_mask <- feature == ''
    feature[empty_string_mask] <- NA
    return(feature)
}

In [14]:
for (feature in empty_means_without) {
    housing_df[,feature] <- replace_empty_with_without(housing_df[,feature])
}

for (feature in empty_means_NA) {
    housing_df[,feature] <- replace_empty_with_NA(housing_df[,feature])
}

In [15]:
count_empty_total()

[1] "Electrical 1"


In [16]:
nan_sums = colSums(is.na(housing_df))
nan_sums[nan_sums > 0]

Electrical 
         1

Following this, we still have a `nan` value that has not been dealt with. As the affected data amounts to less than .1% our our total data, we simply drop the affected row.

In [17]:
housing_df <- na.omit(housing_df)

In [18]:
dim(housing_df)

[1] 1459   81